In [1]:
import numpy as np
import tensorflow as tf
import tools
import metrics
import keras_losses as kl
import keras_models as kms
import bnn_predictions as bnn
import output_tools
import scipy.stats.stats

/home/astrophysics/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
run_ns = range(1,11)
#architecture parameters
num_inputs = 13
num_outputs = 1
layer_sizes = []
m = 506 #total number of records
training_size = 253. / 506.
test_size = 1 - training_size
test_num = int(np.ceil(m * test_size))

In [8]:
#following loops could easily be included into same loop, but currently have them separate for clarity

In [41]:
MPE_test_losses = []
for run_n in run_ns:
    #load data
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter = ',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter = ',')
    #load keras MPE model
    model_r = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/uci/bh_50_slp_" + str(run_n) + ".h5", custom_objects={'sum_of_squared_errors': kl.sum_of_squared_errors})
    #calculate evaluation metrics for keras MPE
    y_pred_r = model_r.predict(x_test)
    y_test, y_pred_r = tools.check_dtypes(y_test, y_pred_r)
    y_test = y_test.reshape(-1,1)
    test_loss_r = metrics.mean_squared_error(y_test, y_pred_r) #averaged over test set size
    MPE_test_losses.append(test_loss_r)

In [43]:
#load in model from first run
x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(1) + ".csv", delimiter = ',')
y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(1) + ".csv", delimiter = ',')
y_test = y_test.reshape(-1,1)
#load keras MPE model
model_r = tf.keras.models.load_model("../../MPE_examples/saved_keras_models/uci/bh_50_slp_" + str(1) + ".h5", custom_objects={'sum_of_squared_errors': kl.sum_of_squared_errors})
#calculate evaluation metrics for keras MPE
y_pred_r = model_r.predict(x_test)

In [44]:
#this is for testing purposes 
#model.evaluate should be used with caution, as if mode contains regularisation, the method may include it
#N.B. SEEMS THAT LOSS TERM DOES INCLUDE IT IN SOME SENSE, BUT METRIC TERM DOESN'T
score_r = model_r.evaluate(x_test, y_test, verbose=0, batch_size=test_num)
print('Test sum of squared loss:', score_r[0])
print('Test mean squared loss:', score_r[1])

('Test sum of squared loss:', 95.2628402709961)
('Test mean squared loss:', 0.37552011013031006)


In [7]:
print tools.get_km_weight_magnitudes(model_r)
model_r.get_weights()

([0.26956007, 7.5576195e-06], 0.26956007)


[array([[-0.04792913],
        [ 0.03673083],
        [-0.05607983],
        [ 0.07248197],
        [-0.03298995],
        [ 0.14594354],
        [-0.03973474],
        [-0.00864891],
        [-0.02216069],
        [-0.04240055],
        [-0.09921065],
        [ 0.0569996 ],
        [-0.14558701]], dtype=float32), array([-7.5576195e-06], dtype=float32)]

In [7]:
#cpp bnn

In [10]:
# bnn expected estimates
bnn_test_losses = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/bh_50_slp_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = y_test.reshape(-1,1)
    bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.mean_squared_error(y_test, by_pred)
    bnn_test_losses.append(b_test_loss)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
# bnn MPE estimates
bnn_MPE_test_losses = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/bh_50_slp_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = y_test.reshape(-1,1)
    bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
    by_pred = bnn_model.MPE_prediction()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    b_test_loss = metrics.mean_squared_error(y_test, by_pred)
    bnn_MPE_test_losses.append(b_test_loss)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
print tools.get_km_weight_magnitudes(bnn_model.model)

[[-0.85722125  2.201045  ]
 [-1.128426   -0.9618036 ]
 [-0.01186933 -0.07161662]
 [-0.19148481  1.4325533 ]
 [-1.3325505  -1.288243  ]
 [ 1.5406443   0.49094737]
 [ 1.111408    2.7693534 ]
 [ 0.07202915  0.6641578 ]
 [-0.30221823  1.0426121 ]
 [-0.5718678   0.97703904]
 [ 1.7841057  -0.41787377]
 [-0.16570689 -0.02360797]
 [-1.1184111  -1.1554649 ]
 [-0.08572011 -0.22775735]]
[-0.9668519 -1.263194 ]
5.810641
1.5907425
6.024451
([5.810641, 1.5907425], 6.024451)


In [42]:
print MPE_test_losses
print np.mean(MPE_test_losses)
print np.std(MPE_test_losses)

[0.3755200976351585, 0.4310342366377511, 0.3055748111011398, 0.4016756274901759, 0.3061682867671988, 0.36535111052988056, 0.4083179007306738, 0.335208736172516, 0.2755627818288049, 0.29718509469210913]
0.35015986835854085
0.05095928137196495


In [11]:
print bnn_test_losses
print np.mean(bnn_test_losses)
print np.std(bnn_test_losses)

[0.28373624897904387, 0.43408530398391826, 0.30772015412517006, 0.4017370011645803, 0.30664926346380283, 0.3650593383390473, 0.4086402663314116, 0.3346326578033962, 0.27587627825660727, 0.2972387508625599]
0.3415375263309538
0.05410025203529773


In [12]:
bnn_evidence = []
bnn_evidence_e = []
for run_n in run_ns:
    stats_file = '../cpp_models/cpp_chains/bh_50_slp_' + str(run_n) + '.stats'
    ev, ev_e = output_tools.read_evidence(stats_file)
    bnn_evidence.append(ev)
    bnn_evidence_e.append(ev_e)
print scipy.stats.stats.pearsonr(bnn_test_losses, bnn_evidence)

(0.28408615368681156, 0.42633279569214677)


In [5]:
#just look at first run from cpp chains
chains_file = '../cpp_models/cpp_chains/bh_50_slp_' + str(1) + '.txt'
x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(1) + ".csv", delimiter=',')
y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(1) + ".csv", delimiter=',')
y_test = y_test.reshape(-1, 1)
bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file)
bnn_model.setup_LL('squared_error')

In [9]:
bnn_model.y_pred_paramnames('bh_50_slp_pred_' + str(1))

In [11]:
bnn_model.y_pred_chains('../cpp_models/cpp_chains/bh_50_slp_pred_' + str(1) + '_' + str(1), test_index = 1)

array([[3.18120668e-154, 7.60750497e+002, 3.63046212e-007,
        9.99999642e-001],
       [7.49849493e-149, 7.36007763e+002, 3.97993386e-001,
        6.02006614e-001],
       [5.00839506e-140, 6.95366406e+002, 9.41447735e-001,
        5.85522838e-002],
       ...,
       [1.56641763e-002, 1.61902841e+001, 9.68318820e-001,
        3.16811278e-002],
       [1.71406913e-002, 1.60101263e+001, 9.20546710e-001,
        7.94532746e-002],
       [2.66659221e-002, 1.51262639e+001, 9.51721430e-001,
        4.82785702e-002]])

In [13]:
# bnn single sh expected estimates
n_stoc = 1
sh_bnn_test_losses = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/bh_50_sh_slp_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = y_test.reshape(-1,1)
    bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file, n_stoc)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    sh_b_test_loss = metrics.mean_squared_error(y_test, by_pred)
    sh_bnn_test_losses.append(sh_b_test_loss)

In [14]:
print sh_bnn_test_losses
print np.mean(sh_bnn_test_losses)
print np.std(sh_bnn_test_losses)

[0.28658966019238963, 0.442737895890976, 0.3111487493247008, 0.4068349450629251, 0.30837865378706514, 0.3647521764876046, 0.4040777403407942, 0.33165227681190756, 0.2770991846583203, 0.29658339345681856]
0.34298546760135024
0.05499786304098376


In [15]:
sh_bnn_evidence = []
sh_bnn_evidence_e = []
for run_n in run_ns:
    stats_file = '../cpp_models/cpp_chains/bh_50_sh_slp_' + str(run_n) + '.stats'
    ev, ev_e = output_tools.read_evidence(stats_file)
    sh_bnn_evidence.append(ev)
    sh_bnn_evidence_e.append(ev_e)
print scipy.stats.stats.pearsonr(sh_bnn_test_losses, sh_bnn_evidence)

(0.248231081666487, 0.4892349039165358)


In [16]:
# bnn input_size number of sh expected estimates
n_stoc = 14
ih_bnn_test_losses = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/bh_50_ih_slp_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = y_test.reshape(-1,1)
    bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file, n_stoc)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    ih_b_test_loss = metrics.mean_squared_error(y_test, by_pred)
    ih_bnn_test_losses.append(ih_b_test_loss)

In [17]:
print ih_bnn_test_losses
print np.mean(ih_bnn_test_losses)
print np.std(ih_bnn_test_losses)

[0.28429470340433793, 0.435421339364113, 0.3087640886837946, 0.4027301822796694, 0.306906237147917, 0.36520873863048126, 0.40657481323777495, 0.3346206292498219, 0.27564281846908745, 0.29704376767109275]
0.341720731813809
0.05410024635854808


In [18]:
ih_bnn_evidence = []
ih_bnn_evidence_e = []
for run_n in run_ns:
    stats_file = '../cpp_models/cpp_chains/bh_50_ih_slp_' + str(run_n) + '.stats'
    ev, ev_e = output_tools.read_evidence(stats_file)
    ih_bnn_evidence.append(ev)
    ih_bnn_evidence_e.append(ev_e)
print  scipy.stats.stats.pearsonr(ih_bnn_test_losses, ih_bnn_evidence)

(0.24299240314787576, 0.4987431473767305)


In [3]:
# bnn single sh single sv expected estimates
n_stoc = 1
n_stoc_var = 1
sh_sv_bnn_test_losses = []
for run_n in run_ns:
    chains_file = '../cpp_models/cpp_chains/bh_50_sh_sv_slp_' + str(run_n) + '.txt'
    x_test = np.genfromtxt("../../data/uci/bh_50_x_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = np.genfromtxt("../../data/uci/bh_50_y_te_" + str(run_n) + ".csv", delimiter=',')
    y_test = y_test.reshape(-1,1)
    bmodel = kms.slp(num_inputs, num_outputs, layer_sizes)
    bnn_model = bnn.bnn_predictor(bmodel, x_test, y_test, chains_file, n_stoc, n_stoc_var)
    by_pred = bnn_model.predictions_expectation()
    y_test, by_pred = tools.check_dtypes(y_test, by_pred)
    sh_sv_b_test_loss = metrics.mean_squared_error(y_test, by_pred)
    sh_sv_bnn_test_losses.append(sh_sv_b_test_loss)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
print sh_sv_bnn_test_losses
print np.mean(sh_sv_bnn_test_losses)
print np.std(sh_sv_bnn_test_losses)

[0.2823283696034877, 0.43543835782854884, 0.3092685092584913, 0.4025503295159994, 0.305891248959838, 0.36636849967547463, 0.40823413737402786, 0.33438807591374864, 0.2751784013806746, 0.29699637811955026]
0.34166423076298413
0.05464523374752586


In [5]:
sh_sv_bnn_evidence = []
sh_sv_bnn_evidence_e = []
for run_n in run_ns:
    stats_file = '../cpp_models/cpp_chains/bh_50_sh_sv_slp_' + str(run_n) + '.stats'
    ev, ev_e = output_tools.read_evidence(stats_file)
    sh_sv_bnn_evidence.append(ev)
    sh_sv_bnn_evidence_e.append(ev_e)
print scipy.stats.stats.pearsonr(sh_sv_bnn_test_losses, sh_sv_bnn_evidence)

(0.2841469577718949, 0.4262294669573053)
